In [77]:
%pip install pandas yfinance tqdm eikon

^C
Traceback (most recent call last):
  File "/Users/christopher/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/christopher/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pip/__main__.py", line 31, in <module>
    sys.exit(_main())
  File "/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/main.py", line 70, in main
    return command.main(cmd_args)
  File "/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 101, in main
    return self._main(args)
  File "/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 223, in _main
    self.handle_pip_version_check(options)
  File "/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/req_command.py", 

In [ ]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm
import numpy as np
import eikon as ek
import requests

# Set Eikon API key
ek.set_app_key('e1df9a44dc6c4206a0599ca61d351c5a92689f71')

In [388]:
input_file = '/Users/christopher/Desktop/Seminar-Advanced-Finance/stocks.csv'

# Read the CSV file
stocks_df = pd.read_csv(input_file)

In [ ]:
# We exclude the 'Financials' from the dataframe
List = ['Health Care',
        'Information Technology',
        'Consumer Discretionary',
        'Consumer Staples',
        'Industrials',
        'Utilities',
        'Materials',
        'Real Estate',
        'Energy',
        'Communication Services']
stocks_df = stocks_df[stocks_df['GICS Sector'].isin(List)]
stocks_df["CIK"] = stocks_df["CIK"].astype(int)

# List to accumulate data frames for each ticker
final_df2 = pd.DataFrame()

In [ ]:
# Use a proper User-Agent header (SEC blocks default Python agents)
url = "https://www.sec.gov/files/company_tickers_exchange.json"
headers = {'User-Agent': 'Mozilla/5.0 (compatible; your_email@example.com)'}
response = requests.get(url, headers=headers)

# Check content before parsing
if response.status_code == 200 and response.text.strip().startswith("{"):
    sec_data = response.json()
else:
    raise ValueError("Failed to fetch SEC data. Status code or content invalid.")

# Parse SEC data
sec_df = pd.DataFrame(sec_data["data"], columns=sec_data["fields"])
sec_df["cik"] = sec_df["cik"].astype(int)

# Merge on CIK
stocks_df = pd.merge(stocks_df, sec_df, left_on="CIK", right_on="cik", how="left")

# Map exchanges to RIC suffixes
exchange_suffix_map = {
    "Nasdaq": ".O",
    "NYSE": ".N",
    "OTC": ".PK",
    "CBOE": ".Z"
}
stocks_df["RIC_Suffix"] = stocks_df["exchange"].map(exchange_suffix_map)
stocks_df["Ticker"] = stocks_df["Symbol"] + stocks_df["RIC_Suffix"]
stocks_df = stocks_df.drop_duplicates(subset="Security", keep="first").reset_index(drop=True)

In [391]:
# Create a list of tickers with the "@ticker" suffix
RICS_list =[Ticker + "@ticker" for Ticker in stocks_df['Ticker'].tolist()]

# Retrieve tickers information using Eikon API
tickers, err = ek.get_data(RICS_list, 'TR.RIC')

/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/eikon/data_grid.py:255: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [ ]:
# If a row's 'RIC' is missing or empty, it fills it with the 'Instrument' value
tickers['RIC'] = tickers.apply(
    lambda row: row['Instrument'] if pd.isna(row['RIC']) or row['RIC'] == '' else row['RIC'],
    axis=1
)

In [ ]:
# Temporarily rename instrument and then merges with stock_df to rename again
tickers.rename(columns={'Instrument': 'Ticker'}, inplace=True)
tickers = pd.merge(tickers, stocks_df, on='Ticker', how='left')
tickers.rename(columns={'Ticker': 'Instrument'}, inplace=True)

In [ ]:
# Pulls U.S. GDP data quarterly, and then calculates the GDP growth rate for each quarter.
USGDP = ek.get_timeseries('aUSGDPCC/CA', 
                       start_date="1999-01-01", 
                       end_date="2025-12-31", 
                       interval="quarterly")
USGDP['Return'] = USGDP['VALUE'] / USGDP['VALUE'].shift(1) - 1

In [396]:
# Define the fields to retrieve for quarterly fundamentals
fields = [
    'TR.TotalEquity(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ).periodenddate',
    'TR.TotalEquity(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.NetIncome(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.TtlLiabShareholderEqty(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.IncomeTaxRatePct(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.TotalReceivablesNet(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.INVNTTOT(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.PPENETTOT(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.Revenue(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.DIVYLDCOMSTOCKISSUEPCTTTM(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.NETINCAFTERMININTR(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.MKTCAP(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.MKTCAP3YRAVG(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.TOTDEBTPCTOFTOTASSETS(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.TOTDEBTPCTOFTOTEQ(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.TOTDEBTPCTOFTOTCAP(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.PPEEXCLROUTANGCAPLEASENETSUPPL(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.RND(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.PRICECLOSEENDPRD(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.PRICETOEPSDILEXCLEXORDNORMTOTTTM(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.EBITDA(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.DEBTTOT(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.TotalAssets(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)'
]

# Define parameters: Scale adjusts the numeric magnitude and Curn sets the currency.
params = {'Scale': 6, 'Curn': 'USD'}

# Initialize an empty DataFrame to store all tickers' fundamental data
stocksinfo_df = pd.DataFrame()

# Iterate over each ticker in the tickers DataFrame using the 'RIC' column
for idx, row in tqdm(tickers.iterrows(), total=tickers.shape[0], desc="Downloading fundamentals"):
    ticker = row['RIC']  # Assuming your tickers DataFrame has a column named 'RIC'
    GICS = row['GICS Sector']
    data, err = ek.get_data(ticker, fields, parameters=params)
    if err:
        print(f"Error retrieving data for {ticker}: {err}")
        continue
    temp_df = pd.DataFrame(data)
    # Add the ticker identifier to the temporary DataFrame
    temp_df['Ticker'] = ticker
    temp_df['GICS Sector'] = GICS
    stocksinfo_df = pd.concat([stocksinfo_df, temp_df], ignore_index=True)

  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='i

Error retrieving data for BF.B.N: [{'code': 412, 'col': 1, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 2, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 3, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 4, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 5, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 6, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 7, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 8, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 9, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 10, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 11,

  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='i

In [ ]:
import pandas as pd
import yfinance as yf

# Download daily VIX data
vix_data = yf.download("^VIX", start="1999-01-01", end="2025-12-31")

# Resample to quarterly using the last closing value of each quarter
vix_quarterly = vix_data[['Close']].resample('Q').last()

# Reset index to ensure it's a single-level DataFrame
vix_quarterly = vix_quarterly.reset_index()

# Convert 'Date' column to quarterly periods
vix_quarterly['Period'] = vix_quarterly['Date'].dt.to_period('Q')

# FIX: Flatten MultiIndex if necessary
if isinstance(vix_quarterly.columns, pd.MultiIndex):
    vix_quarterly.columns = [col[0] for col in vix_quarterly.columns]

# Keep only 'Period' and 'Close', and rename 'Close' to 'VIX_Close'
vix_quarterly = vix_quarterly[['Period', 'Close']].rename(columns={'Close': 'VIX_Close'})

# Calculate VIX Growth Rate (quarter-over-quarter change)
vix_quarterly['VIX_Growth'] = vix_quarterly['VIX_Close'].pct_change()

# Convert 'Period End Date' in stocksinfo_df to datetime and extract quarterly period
stocksinfo_df['Period'] = pd.to_datetime(stocksinfo_df['Period End Date']).dt.to_period('Q')

# Convert Period to string in both DataFrames to ensure matching format
stocksinfo_df['Period'] = stocksinfo_df['Period'].astype(str)
vix_quarterly['Period'] = vix_quarterly['Period'].astype(str)

# Merge stocksinfo_df with vix_quarterly on 'Period'
stocksinfo_df = pd.merge(stocksinfo_df, vix_quarterly, on='Period', how='left')

[*********************100%***********************]  1 of 1 completed
/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_94536/646556795.py:8: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  vix_quarterly = vix_data[['Close']].resample('Q').last()


In [ ]:
# Saves the dataframe
stocksinfo_df.to_csv('/Users/christopher/Desktop/Seminar-Advanced-Finance/stocksinfo_data.csv', index=False)

In [ ]:
# Convert 'Period End Date' in stocksinfo_df to datetime if it isn’t already
stocksinfo_df['Period End Date'] = pd.to_datetime(stocksinfo_df['Period End Date'])
# Create a new column representing the quarter period
stocksinfo_df['Period'] = stocksinfo_df['Period End Date'].dt.to_period('Q')

# Ensure USGDP's date column is datetime. If USGDP’s date is the index, you can reset the index:
if 'Date' not in USGDP.columns:
    USGDP = USGDP.reset_index()
USGDP['Date'] = pd.to_datetime(USGDP['Date'])
# Create a quarterly period column for USGDP as well
USGDP['Period'] = USGDP['Date'].dt.to_period('Q')

# Now merge on the 'Period' column
merged_df = pd.merge(stocksinfo_df, USGDP, on='Period', how='left')
stocksinfo_df = merged_df

In [ ]:
# Remove rows where 'AdjustedDate' is NaT or has a year before 2000
stocksinfo_df = stocksinfo_df[stocksinfo_df['Period End Date'].notnull() & (stocksinfo_df['Period End Date'].dt.year >= 2000)]

In [ ]:
# Compute Debt Ratio:
# It measures how much of a company's assets are financed by debt.
stocksinfo_df["Debt Ratio"] = (
    (stocksinfo_df["Total Liabilities And Shareholders' Equity"] - stocksinfo_df["Total Equity"]) /
    stocksinfo_df["Total Assets"]
)

# Compute ROE:
# ROE = Net Income Incl Extra Before Distributions / Total Equity
stocksinfo_df["ROE"] = (
    stocksinfo_df["Net Income Incl Extra Before Distributions"] / stocksinfo_df["Total Equity"]
)

# Compute Asset Tangibility:
# Asset Tangibility = (Property Plant & Equipment - Net - Total + Inventories - Total) / Total Assets
stocksinfo_df["Property Plant & Equipment - Net - Total"] = stocksinfo_df["Property Plant & Equipment - Net - Total"].fillna(0)
stocksinfo_df["Asset Tangibility"] = (
    (stocksinfo_df["Property Plant & Equipment - Net - Total"] + stocksinfo_df["Inventories - Total"]) /
    stocksinfo_df["Total Assets"]
)

# Compute Tobins Q:
# Shows how highly the market values the firm's assets compared to their book value.
stocksinfo_df["TobinsQ"] = (
    (stocksinfo_df["Market Capitalization"]) /
    stocksinfo_df["Total Assets"]
)

# Compute Research and Development intensity:
# It measures how much of the company’s assets are invested in R&D activities
stocksinfo_df["RnDintensity"] = (
    (stocksinfo_df["Research & Development Expense"]) /
    stocksinfo_df["Total Assets"]
)

# Compute PE Ratio:
# Shows how much investors are willing to pay for each unit of earnings
stocksinfo_df["PEratio"] = (
    (stocksinfo_df["Price Close (End of Period)"]) /
    stocksinfo_df["Price to EPS Diluted excl Exord - Norm - Total, TTM"]
)

# Compute Size
# It measures firm size based on the log of total assets
# which is common in finance and economics to handle large variations between companies
stocksinfo_df['Size'] = np.log(stocksinfo_df['Total Assets'])

stocksinfo_df.rename(columns={'Dividend Yield - Common - Net - Issue - %, TTM': 'DPR'}, inplace=True)
stocksinfo_df.rename(columns={'Net Income after Minority Interest': 'Net Income'}, inplace=True)
stocksinfo_df.rename(columns={'VALUE': 'GDP'}, inplace=True)
stocksinfo_df.rename(columns={'Return': 'GDP Growth'}, inplace=True)
stocksinfo_df.rename(columns={'Earnings before Interest Taxes Depreciation & Amortization': 'EBITDA'}, inplace=True)

/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_94536/1035504047.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksinfo_df["Debt Ratio"] = (
/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_94536/1035504047.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksinfo_df["ROE"] = (
/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_94536/1035504047.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [ ]:
import pandas_datareader.data as web
import datetime

# Define date range
start = datetime.datetime(1999, 1, 1)
end = datetime.datetime(2025, 12, 31)

# Get U.S. Fed Funds Rate from FRED
interest_rate_data = web.DataReader("FEDFUNDS", "fred", start, end)

# Convert index to a column
interest_rate_data.reset_index(inplace=True)

# Convert 'DATE' column to quarterly period
interest_rate_data['Period'] = interest_rate_data['DATE'].dt.to_period('Q')

# Rename columns
interest_rate_data.rename(columns={"FEDFUNDS": "Interest Rate"}, inplace=True)

# Sort by DATE in descending order to ensure the latest is first
interest_rate_data = interest_rate_data.sort_values(by="DATE", ascending=False)

# Keep only the latest entry for each Period
interest_rate_quarterly = interest_rate_data.drop_duplicates(subset=["Period"], keep="first")

# Reset index after filtering
interest_rate_quarterly = interest_rate_quarterly.reset_index(drop=True)

# Ensure both Period columns are in the same format (string)
stocksinfo_df['Period'] = stocksinfo_df['Period'].astype(str)
interest_rate_quarterly['Period'] = interest_rate_quarterly['Period'].astype(str)

# Merge stocksinfo_df with interest_rate_quarterly on 'Period'
stocksinfo_df = pd.merge(stocksinfo_df, interest_rate_quarterly, on='Period', how='left')

/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_94536/2459230991.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksinfo_df['Period'] = stocksinfo_df['Period'].astype(str)


In [ ]:
import numpy as np
import statsmodels.api as sm

# Replace any +inf or -inf with NaN in your entire panel_data
final_merged_df = stocksinfo_df.replace([np.inf, -np.inf], np.nan)

In [ ]:
# Create lagged debt ratio (1 period back) for each firm
final_merged_df['DebtRatio_Lag'] = final_merged_df.groupby('Instrument')['Debt Ratio'].shift(1)

# Create lagged VIX growth (1 period back) for each firm
final_merged_df['VIX_Growth_Lag'] = final_merged_df.groupby('Instrument')['VIX_Growth'].shift(1)

# Create lagged VIX growth (2 periods back) for each firm
final_merged_df['VIX_Growth_Lag2'] = final_merged_df.groupby('Instrument')['VIX_Growth'].shift(2)

# Create interaction term between debt ratio and VIX growth
final_merged_df['Debt_VIX_Interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['VIX_Growth']

# Take the log(1 + Tobin's Q) to smooth values
final_merged_df['Log_TobinsQ'] = np.log1p(final_merged_df['TobinsQ'])

# Create interaction between debt ratio and firm size
final_merged_df['Debt_Size_Interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['Size']

# Take the log(1 + Debt Ratio) to handle zeros and reduce skewness
final_merged_df['Log_DebtRatio'] = np.log1p(final_merged_df['Debt Ratio'])

# Interaction between logged debt ratio and size
final_merged_df['Log_Debt_Size_Interaction'] = final_merged_df['Log_DebtRatio'] * final_merged_df['Size']

# Create a squared term for debt ratio
final_merged_df['DebtRatio_sq'] = final_merged_df['Debt Ratio'] ** 2

# Fill missing R&D intensity values with 0
final_merged_df["RnDintensity"] = final_merged_df["RnDintensity"].fillna(0)

# Create a squared term for Return on Equity (ROE)
final_merged_df['ROE_sq'] = final_merged_df['ROE'] ** 2

# Sort the dataset by firm and date
final_merged_df = final_merged_df.sort_values(by=['Instrument', 'Period End Date'])

# Create lagged P/E ratio (1 period back) for each firm
final_merged_df['PEratio_lag1'] = final_merged_df.groupby('Instrument')['PEratio'].shift(1)

# Create interaction between debt ratio and lagged P/E ratio
final_merged_df['Debt_PE_lagged_interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['PEratio_lag1']

# Calculate median debt ratio by industry and date
final_merged_df['Industry_Median_Leverage'] = final_merged_df.groupby(['GICS Sector', 'Period End Date'])['Debt Ratio'].transform('median')

# Take the log(1 + Net Income) to normalize the income variable
final_merged_df['Log_NetIncome'] = np.log1p(final_merged_df['Net Income'])

# Convert GDP growth to percentage form
final_merged_df['GDP_Growth_pct'] = final_merged_df['GDP Growth']*100

# Create a squared term for firm size
final_merged_df['Size_sq'] = final_merged_df['Size'] ** 2

# Create interaction between debt ratio and P/E ratio
final_merged_df['Debt_PE_interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['PEratio']

/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
# Create a constant to shift Net Income so all values become positive
constant_shift = abs(final_merged_df['Net Income'].min()) + 1

# Create a new variable by taking the natural log of the shifted Net Income
final_merged_df['Log_NetIncome_Shifted'] = np.log(final_merged_df['Net Income'] + constant_shift)

In [ ]:
# Create a constant to shift EBITDA so all values become positive
constant_shift = abs(final_merged_df['EBITDA'].min()) + 1

# Create a new variable by taking the natural log of the shifted EBITDA
final_merged_df['Log_EBITDA'] = np.log(final_merged_df['EBITDA'] + constant_shift)

# Create interaction between debt ratio and logged EBITDA
final_merged_df['Debt_LogEBITDA_Interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['Log_EBITDA']

# Create interaction between debt ratio and raw (non-logged) EBITDA
final_merged_df['Debt_EBITDA_Interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['EBITDA']

In [415]:
panel_data = final_merged_df.set_index(['Instrument', 'Period End Date'])
xx = panel_data[['Log_TobinsQ','Debt Ratio' , 'ROE', 'Asset Tangibility', 'Size', 'DPR', 'Log_NetIncome_Shifted', 'GDP_Growth_pct', 'VIX_Growth', 'RnDintensity', 'PEratio', 'Interest Rate', 'EBITDA', 'Industry_Median_Leverage']]

In [ ]:
# xx is the dataframe
summary_stats = xx.describe().T  # Transpose to match LaTeX table format

# Round numbers for better readability
summary_stats = summary_stats[['mean', 'std', 'min', 'max']].round(3)

# Rename standard deviation column
summary_stats.rename(columns={'std': 'sd'}, inplace=True)

# Add the observation row properly
summary_stats.loc["Observations"] = [f"{xx.shape[0]:,}", "", "", ""]

# Convert to LaTeX with proper formatting
latex_code = summary_stats.to_latex(
    column_format="lcccc",  # Ensures alignment
    escape=False,
    caption="Deskriptiv statistik",
    label="tab:summary"
)

# Ensure Observations row is well-placed
latex_code = latex_code.replace(
    r"\bottomrule",
    f"Observations & {xx.shape[0]:,} & & & \\\\\n\ bottomrule"
)

# Save to file
with open("summary_statistics.tex", "w") as f:
    f.write(latex_code)

print(latex_code)  # Print LaTeX table


\begin{table}
\caption{Deskriptiv statistik}
\label{tab:summary}
\begin{tabular}{lcccc}
\toprule
 & mean & sd & min & max \\
\midrule
Log_TobinsQ & 0.924931 & 0.538040 & 0.000000 & 4.442504 \\
Debt Ratio & 0.610845 & 0.266398 & 0.024607 & 4.481232 \\
ROE & 0.031530 & 1.466537 & -104.611111 & 140.339244 \\
Asset Tangibility & 0.406615 & 0.285758 & -24.315227 & 2.899001 \\
Size & 9.267586 & 1.527731 & 0.038259 & 13.649442 \\
DPR & 1.437827 & 2.034194 & -67.398119 & 52.677279 \\
Log_NetIncome_Shifted & 10.358648 & 0.068819 & 0.000000 & 11.971969 \\
GDP_Growth_pct & 0.353322 & 1.311970 & -7.970840 & 7.775070 \\
VIX_Growth & 0.057316 & 0.444818 & -0.460661 & 2.885341 \\
RnDintensity & 0.006754 & 0.014762 & -0.106468 & 0.394303 \\
PEratio & 4.690825 & 101.970460 & 0.000370 & 18978.128940 \\
Interest Rate & 1.880220 & 2.001812 & 0.070000 & 6.530000 \\
EBITDA & 979.601586 & 2296.111699 & -89284.000000 & 45912.000000 \\
Industry_Median_Leverage & 0.600957 & 0.163084 & 0.124984 & 4.470956 \\
Obs

In [414]:
final_merged_df.to_csv('/Users/christopher/Desktop/Seminar-Advanced-Finance/finaldata.csv', index=False)